# Extraction des connaissances des données

Les données choisies proviennent d'un dataset fournis par audi. Ce sont des données extraite des entrainements de leurs voitures autonome. 
https://www.a2d2.audi/a2d2/en/dataset.html

Il est composé de:
* Données lidar
* Vidéos des caméras embarqués
* Bounding box 3D détectées par leurs véhicule
* Signaux du bus CAN

Nous avons choisis les signaux CAN car ce sont les données les plus simple à analyser et les plus légère (le dataset complet pèse 2.3To).
Les données sont fournis dans un fichier json contenant toutes les informations. Nous allons tout d'abord extraire les données dans un fichier csv simplifier en extrayant et formatant les informations suivantes:
* Pression de la pédale d'accèlération
* Pression de la pédale de freinage
* Distances retournées par les capteurs avant et arrière droite et gauche
* Vitesse du véhicule

In [1]:
import json
import csv

## Préparation des données

### Lecture des données brut

In [2]:
with open('bus_signals.json', 'r') as input_file:
    initial_datas = json.load(input_file)
print(initial_datas.keys())

dict_keys(['accelerator_pedal', 'accelerator_pedal_gradient_sign', 'angular_velocity_omega_x', 'angular_velocity_omega_y', 'angular_velocity_omega_z', 'brake_pressure', 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right', 'latitude_degree', 'latitude_direction', 'longitude_degree', 'longitude_direction', 'pitch_angle', 'roll_angle', 'steering_angle_calculated', 'steering_angle_calculated_sign', 'vehicle_speed'])


In [3]:
keys = ['accelerator_pedal', 'brake_pressure', 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right', 'vehicle_speed']
units = [initial_datas[key]['unit'] if not initial_datas[key]['unit'] is None else 'CentiMeter' for key in keys]

### Extraction de formatage des données initiales

Chaque données est fournis avec un timestamp d'une précision de 17 digits. Nous allons diminuer cette précision afin de pouvoir fusionner des données et obtenir des lignes complètes sans donnée manquante.

In [4]:

merged_datas = dict()
for key in keys:
    values = initial_datas[key]['values'].copy()
    for i in range(len(values)):
        timestamp = round(values[i][0]/1e4)
        if not timestamp in merged_datas:
            merged_datas[timestamp] = dict(accelerator_pedal=None, brake_pressure=None, distance_pulse_front_left=None, distance_pulse_front_right=None, distance_pulse_rear_left=None, distance_pulse_rear_right=None, vehicle_speed=None)
        if merged_datas[timestamp][key] is None:
            merged_datas[timestamp][key] = values[i][1]


### Création du fichier csv

Nous sauvegardons les données dans un fichier csv afin de pouvoir les recharger plus facilement, et au besoin les ouvrir dans R

In [5]:
timestamps = sorted(list(merged_datas.keys()))
list_data = []
for timestamp in timestamps:
    row = [timestamp] + [merged_datas[timestamp][key] for key in keys]
    if not None in row:
        list_data.append(row)

In [6]:
with open('bus_signals.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['timestamp'] + keys)
    csv_writer.writerow(['MicroSecond'] + units)
    for i in range(len(list_data)):
        csv_writer.writerow(list_data[i])

## Analyse des données